In [15]:
import iris
import numpy as np
from iris.coords import DimCoord
from iris.coord_systems import TransverseMercator,GeogCS
from iris.cube import Cube
from cf_units import Unit
import cf_units
import os
import glob
from pyproj import Proj, transform
import sys
import warnings
import multiprocessing as mp

warnings.filterwarnings("ignore")

ems_hourly = ['01', '04', '06', '07', '08', '09', '10', '11', '12', '13', '15']
ems_30mins = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc013',  'bc015',  'bc016', 'bc017', 'bc018', 'bc012']
yrs_range = '1980_2001'

In [16]:
##########################################################################################
#########################################################################################
# Define variables and set up environment
##########################################################################################
##########################################################################################
root_fp = "/nfs/a319/gy17m2a/"
os.chdir(root_fp)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'PhD/Scripts/DataProcessing/Regridding')
from Regridding_functions import *
# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
# from Spatial_geometry_functions import *

# Load UKCP18 12km model data to use in regriddding
file_model_12km=f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/12km/01/{yrs_range}/pr_rcp85_land-rcm_uk_12km_01_day_19801201-19901130.nc'
cube_12km=iris.load_cube(file_model_12km)

file_model_2_2km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_model_2_2km =iris.load_cube(file_model_2_2km)

In [17]:
##################################################################
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})
cube_12km_trimmed_to_leeds =  trim_to_bbox_of_region_obs(cube_12km, leeds_at_centre_gdf)

## Regrid hourly UKCP18 data to 12km - DONE

In [ ]:
for em in ems_hourly:
    print(em)
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em}/1980_2001/")
    output_fp = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_regridded_12km/{em}/NearestNeighbour/1980_2001/"
    if not os.path.isdir(output_fp):
        os.makedirs(output_fp)
    for filename in glob.glob("*"):     
            print(filename)
            cube = iris.load(filename)[0]
            # Linear interpolation
            #reg_cube_lin =cube.regrid(cube_model,iris.analysis.Linear())      
            # Nearest neighbour
            reg_cube_nn =cube.regrid(cube_12km,iris.analysis.Nearest())   
            # Save 
            iris.save(reg_cube_nn, output_fp +  f"rg_{filename}")     

01
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19930701-19930730.nc


## Regrid 30 mins UKCP18 data to 12km - DONE
For this need to use version of the regridding cube which is trimmed to the leeds-at-centre region, as the 30 mins data is already clipped to this extent

In [21]:
for em in ems_30mins:
    print(em)
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km/{em}/2001_2020/")
    output_fp = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_regridded_12km/{em}/NearestNeighbour/2001_2020/"
    if not os.path.isdir(output_fp):
        os.makedirs(output_fp)
    for filename in glob.glob("*"):     
            # print(filename)
            cube = iris.load(filename)[0]
            # Linear interpolation
            #reg_cube_lin =cube.regrid(cube_model,iris.analysis.Linear())      
            # Nearest neighbour
            reg_cube_nn =cube.regrid(cube_12km_trimmed_to_leeds,iris.analysis.Nearest())   
            # print(reg_cube_nn.shape)
            # Save 
            iris.save(reg_cube_nn, output_fp +  f"rg_{filename}")   

bc005
bc006
bc007
bc009
bc010
bc011
bc013
bc015
bc016
bc017
bc018
bc012


## Regrid NIMROD - DONE

In [14]:
for year in range(2011,2012):
    print(year)
    # Change directory to be for correct year
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/{year}")
    # Define filepaths to save files to
    output_dir_12km = f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_12km/NearestNeighbour/{year}/"
    output_dir_2_2km = f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_2.2km/NearestNeighbour/{year}/"
    # Create these directories if they don't exist already
    if not os.path.isdir(output_dir_12km):
        os.makedirs(output_dir_12km)
    if not os.path.isdir(output_dir_2_2km):
        os.makedirs(output_dir_2_2km)
    # Loop through all the diles in the 1km folder    
    for filename in sorted(glob.glob("*")):
        if filename != 'metoffice-c-band-rain-radar_uk_20090728_30mins.nc':
            print(filename)
            # Create version of filename specifying it is regridded
            filename_to_save_to = f"rg_{filename}"

            # Check if this regridded file exists, and if not create it
            # Don't want to load the cube twice unnecessarily, so if we load it for 12km, then make a flag to tell us it's
            # already loaded and then use this for 2.2km

            # 12km regridding
            if os.path.isfile(output_dir_12km + filename_to_save_to):
                print("File already exists")
            if not os.path.isfile(output_dir_12km + filename_to_save_to):
                print('Making file 12km')
                cube = iris.load(filename)[0]
                loaded_cube=True
                # Nearest neighbour
                reg_cube_nn =cube.regrid(cube_12km,iris.analysis.Nearest())    
                # Save 
                iris.save(reg_cube_nn, output_dir_12km + filename_to_save_to)

            # 2.2km regridding
            if os.path.isfile(output_dir_2_2km + filename_to_save_to):
                print("File already exists")
            if not os.path.isfile(output_dir_2_2km + filename_to_save_to):
                print('Making 2.2km file')
                if loaded_cube == False:
                    cube = iris.load(filename)[0]
                # Nearest neighbour
                reg_cube_nn =cube.regrid(cube_model_2_2km,iris.analysis.Nearest())    
                print("Regridded")
                # Save 
                iris.save(reg_cube_nn, output_dir_2_2km + filename_to_save_to)    

            loaded_cube=False


2006
metoffice-c-band-rain-radar_uk_20060601_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060602_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060603_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060604_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060605_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060606_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060607_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060608_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060609_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060610_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20060611_30mins.nc
File already exists
File already exists
metof

File already exists
metoffice-c-band-rain-radar_uk_20070611_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070612_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070613_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070614_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070615_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070616_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070617_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070618_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070619_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070620_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20070621_30mins.nc
File already exists
File alrea

File already exists
metoffice-c-band-rain-radar_uk_20080724_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080725_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080726_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080727_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080728_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080729_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080730_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080731_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080801_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080802_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20080803_30mins.nc
File already exists
File alrea

metoffice-c-band-rain-radar_uk_20100601_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100602_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100603_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100604_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100605_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100606_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100607_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100608_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100609_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100610_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20100611_30mins.nc
File already exists
File already exists
metoffice-

metoffice-c-band-rain-radar_uk_20110601_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110602_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110603_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110604_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110605_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110606_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110607_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110608_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110609_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110610_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20110611_30mins.nc
File already exists
File already exists
metoffice-

File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120704_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120705_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120706_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120707_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120708_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120709_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120710_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120711_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120712_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120713_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20120714_30mins.nc
File alrea

File already exists
metoffice-c-band-rain-radar_uk_20130801_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130802_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130803_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130804_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130805_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130806_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130807_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130808_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130809_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130810_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20130811_30mins.nc
File already exists
File alrea

File already exists
metoffice-c-band-rain-radar_uk_20140817_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140818_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140819_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140820_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140821_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140822_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140823_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140824_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140825_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140826_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20140827_30mins.nc
File already exists
File alrea

File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160610_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160611_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160612_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160613_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160614_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160615_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160616_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160617_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160618_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160619_30mins.nc
File already exists
File already exists
metoffice-c-band-rain-radar_uk_20160620_30mins.nc
File alrea

KeyboardInterrupt: 

## Regrid CEH-GEAR

In [31]:
# os.chdir("/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/OriginalFormat/")
# filenames =[ ]
# for filename in glob.glob("*"):
#     filenames.append(filename)
# len(filenames)

In [ ]:
os.chdir("/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/OriginalFormat/")
for filename in glob.glob("*")
    print(filename)
    cube = iris.load(filename)[0]
    
    # Filename to save reformatted cube to
    filename_to_save_to_reformat = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/rf_{filename}"
    # Filename to save 2.2km regridded cube to -- linear
    filename_to_save_to_lin_2_2km = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_2.2km/LinearRegridding/rg_{filename}"
    # Filename to save 2.2km regridded cube to -- nearest neighbour
    filename_to_save_to_nn_2_2km = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_2.2km/NearestNeighbour/rg_{filename}"
      # Filename to save 12km regridded cube to -- nearest neihgoubt
    #filename_to_save_to_lin_12km = f"/nfs/a319/gy17m2a/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/LinearRegrdding/rg_{filename}"
    filename_to_save_to_nn_12km = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/NearestNeighbour/rg_{filename}"
    
    ###### Open dataset with Xarry and reformat
    xr_ds=xr.open_dataset(filename)
    # Convert to cube in the correct format and save
    cube=make_bng_cube(xr_ds,'rainfall_amount')
    iris.save(cube, filename_to_save_to_reformat)
    
    #### Regrid observations onto 2.2km model grid
    # Lienar interpolation
    reg_cube_lin_2_2km =cube.regrid(cube_model_2_2km,iris.analysis.Linear())      
    # nearest neighbour
    reg_cube_nn_2_2km =cube.regrid(cube_model_2_2km,iris.analysis.Nearest()) 
    # Save 
    iris.save(reg_cube_lin_2_2km, filename_to_save_to_lin_2_2km)
    iris.save(reg_cube_nn_2_2km, filename_to_save_to_nn_2_2km)
    
    #### Regrid observations onto 12km model grid
    # Linear interpolation
    #reg_cube_lin_12km =cube.regrid(cube_12km,iris.analysis.Linear())      
    # nearest neighbour
    reg_cube_nn_12km =cube.regrid(cube_12km,iris.analysis.Nearest()) 
    # Save 
    #iris.save(reg_cube_lin_12km, filename_to_save_to_lin_12km)
    iris.save(reg_cube_nn_12km, filename_to_save_to_nn_12km)